<a href="https://colab.research.google.com/github/plchuaa/gordonchu/blob/master/linkedin_text_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
import os
import json
import re
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

Preprocessing by get text from json by some simple cleaning and store at a dataframe for later use

In [0]:
def text_cleaning(text):
    if len(text) > 0:
        text = text[0]
        text = re.sub(r"\\n", " ", text)
        # text = re.sub(r"\.\.\.", " ", text)
        text = re.sub(r"\.+\.+", " ", text)
        text = re.sub(r"\s{2,}", " ", text)
        text = remove_emoji(text)
        text = remove_url(text)
        text = remove_newline(text)
    else:
        text = ""
    return text


def get_name(json_dictionairy):
    data = json_dictionairy["current"][0]
    name = re.findall(r'(?:\n +)([\S\s]+)(?:\n +Status is)', data)
    name = text_cleaning(name)
    return name


def get_info(json_dictionairy):
    data = json_dictionairy["current"][0]
    info = re.findall(r'(?:See contact info(?:\n +)*[a-z0-9 A-Z+]+(?:\n +)*)([\s\S]+)', data)
    info = text_cleaning(info)
    return info


def get_summary(json_dictionairy):
    data = json_dictionairy["summary"]
    summary = text_cleaning(data)
    return summary


def list_cleaning(data):
    lists = []
    for exp in data:
        if isinstance(exp, list):
            for e in exp:
                lists.append(text_cleaning([e]))
        else:
            lists.append(text_cleaning([exp]))

    return (" ").join(lists) if len(lists) > 0 else ""


def get_experience(json_dictionairy):
    data = json_dictionairy["experience"]
    experience = list_cleaning(data)
    return experience


def get_education(json_dictionairy):
    data = json_dictionairy["education"]
    education = list_cleaning(data)
    return education


def get_license(json_dictionairy):
    data = json_dictionairy["licenses"]
    licenses = list_cleaning(data)
    return licenses


def get_skill(json_dictionairy):
    data = json_dictionairy["skills"]
    skills = list_cleaning(data)
    return skills


def get_tech(json_dictionairy):
    data = json_dictionairy["tech"]
    tech = list_cleaning(data)
    return tech


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_url(text):
    text = re.sub(r'[a-zA-z]+://[^\s]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'[a-zA-Z]+(\.[a-zA-Z]+)+', '', text, flags=re.MULTILINE)
    return text


def remove_newline(text):
    text = re.sub(r"\n", " ", text)
    return text


# Text Preprocessing(remove stopwords,punctuations).
nlp = spacy.load('en_core_web_sm')
# build a list of stopwords to use to filter
stopwords = list(STOP_WORDS)
stopwords.append("see more")



In [3]:
import os
from google.colab import drive

# Get data from drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

profile_df  = pd.DataFrame(columns=["profile_name","profile_detail", "profile_info","profile_summary","profile_experience",
                                         "profile_education","profile_license","profile_skill","profile_tech"])

with os.scandir('./linkedin/') as entries:
    for each in entries:
        file = open(each, 'r', encoding="utf-8")
        profile_data = json.load(file)

        profile_name = get_name(profile_data)
        profile_info = get_info(profile_data)
        profile_summary = get_summary(profile_data)
        profile_experience = get_experience(profile_data)
        profile_education = get_education(profile_data)
        profile_license = get_license(profile_data)
        profile_skill = get_skill(profile_data)
        profile_tech = get_tech(profile_data)
        profile_detail = profile = profile_info + profile_summary + profile_experience + profile_education + profile_license + \
                        profile_skill + profile_tech

        profile_to_row = {}
        profile_to_row["profile_name"] = profile_name
        profile_to_row["profile_info"] = profile_info
        profile_to_row["profile_summary"] = profile_summary
        profile_to_row["profile_experience"] = profile_experience
        profile_to_row["profile_education"] = profile_education
        profile_to_row["profile_skill"] = profile_skill
        profile_to_row["profile_license"] = profile_license
        profile_to_row["profile_tech"] = profile_tech
        profile_to_row["profile_detail"] = profile_detail

        profile_df = profile_df.append(profile_to_row, ignore_index=True)


In [5]:
profile_df

,profile_name,profile_detail,profile_info,profile_summary,profile_experience,profile_education,profile_license,profile_skill,profile_tech
0,Abby Tang,Business Strategist skilled in Joint Marketing...,Business Strategist skilled in Joint Marketing...,Business Strategist skilled in Joint Marketin...,"APCJ Director, Cloud Field Operations Company...","University of California, Irvine Degree Name ...",,Business Alliances Go-to-market Strategy ...,
1,Ai Shinozaki,My nick name is Ai Shinozaki. I am a PG Girl S...,My nick name is Ai Shinozaki. I am a PG Girl S...,My nick name is Ai Shinozaki. I am a PG Girl ...,PG Model Company Name ABC PG Model Dates Empl...,,,Target Orientation Reading Comprehension ...,Other Skills Sexy PG Show Sexy Girl Face & Bo...
2,Dr Alan Chow MIET MIEEE MAUSIMM,Founder of a disruptive project with strong Gl...,Founder of a disruptive project with strong Gl...,Founder of a disruptive project with strong G...,"Chairman, Investment Committee Company Name C...",Renmin University of China Degree Name Doctor...,,Investments Private Equity Corporate Fina...,Industry Knowledge Equities See 25 endorsemen...
3,Abel Cheung,Administrator in a variety of coorporate netwo...,Administrator in a variety of coorporate netwo...,Administrator in a variety of coorporate netw...,Network Architect Company Name Next Media Ani...,AccessData BootCamp & FTK training Field Of S...,Novell Certified Linux Administrator (Novell ...,Linux Apache Ubuntu,
4,ai autodata,Managing Director Company Name,,,Managing Director Company Name,,,,
...,...,...,...,...,...,...,...,...,...
95,Xin LI,I am a second-year . student at CUHK. Prior to...,I am a second-year . student at CUHK. Prior to...,I am a second-year . student at CUHK. Prior t...,"Research Intern, NLP Group@Tencent AI Lab Com...",Field Of Study Faculty of Engineering Dates ...,,,
96,Wong Loretta,Company Name Google Total Duration 4 yrs 3 m...,,,Company Name Google Total Duration 4 yrs 3 mo...,The University of Hong Kong Degree Name Bache...,,Marketing Strategy Search Engine Technology...,Industry Knowledge Business Strategy See 3 en...
97,Worldmoney Blockchain News,Company Name Worldmoney Blockchain Financial...,,,Company Name Worldmoney Blockchain Financial ...,,,,
98,Wayne Liu,Technology professional with 10 years experien...,Technology professional with 10 years experien...,Technology professional with 10 years experie...,Technology Specialist Company Name 3Revolutio...,,,Digital Printing Color Management Printer...,Industry Knowledge R&D See 2 endorsements for...


Possible direction:
1. further split the profile into parts(info + summary, exp, tech+skill+license)

2. modify ways to calculate similarity


In [0]:
# spacy tokenizer to remove stopwords and punctuation, lemma words
def spacy_tokenizer(text):
    parser = English()
    mytokens = parser(text)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords]
    mytokens = [word for word in mytokens if word.isalpha() or word.isnumeric() or len(word)>1 ]
    # mytokens = [word for word in mytokens if word not in string.punctuation]
    return mytokens

In [7]:
example1 = profile_df["profile_detail"][0]
example1 = spacy_tokenizer(example1)
print(example1) # ['ray', 'liew', 'data', 'ai', 'solution', 'specialist', 'microsoft', 'hong', 'kong', 'responsible', 'driving', 'commercial', 'data', 'ai', 'scenarios', 'includes', 'advanced', 'forecasting', 'predictive', 'maintenance', 'customer', '360', 'different', 'sectors', 'hong', 'kong', 'prior', 'current', 'role', 'account', 'manager', 'meltwater', 'news', 'social', 'media', 'analytics', 'company', 'managed', 'portfolio', '65', 'clients', 'helping', 'better', 'informed', 'decisions', 'based', 'insights', 'online', 'data', 'ray', 'spent', 'half', 'life', 'abroad', 'having', 'studied', 'hong', 'kong', 'singapore', 'leverages', 'diverse', 'background', 'malaysian', 'roots', 'enable', 'digital', 'transformation', 'technology', 'geek', 'ray', 'strong', 'passion', 'fitness', 'sports', 'preaches', 'concept', 'integrating', 'fitness', 'lifestyle', 'peers', 'juniors', 'ray', 'holds', 'double', 'major', 'global', 'business', 'information', 'systems', 'hong', 'kong', 'university', 'science', 'technology', 'head', 'student', 'ambassador', 'time', 'university', 'ray', 'wen', 'jie', 'summary', 'ray', 'liew', 'data', 'ai', 'solution', 'specialist', 'microsoft', 'hong', 'kong', 'responsible', 'driving', 'commercial', 'data', 'ai', 'scenarios', 'includes', 'advanced', 'forecasting', 'predictive', 'maintenance', 'customer', '360', 'different', 'sectors', 'hong', 'kong', 'prior', 'current', 'role', 'account', 'manager', 'meltwater', 'news', 'social', 'media', 'analytics', 'company', 'managed', 'portfolio', '65', 'clients', 'helping', 'better', 'informed', 'decisions', 'based', 'insights', 'online', 'data', 'ray', 'spent', 'half', 'life', 'abroad', 'having', 'studied', 'hong', 'kong', 'singapore', 'leverages', 'diverse', 'background', 'malaysian', 'roots', 'enable', 'digital', 'transformation', 'technology', 'geek', 'ray', 'strong', 'passion', 'fitness', 'sports', 'preaches', 'concept', 'integrating', 'fitness', 'lifestyle', 'peers', 'juniors', 'ray', 'holds', 'double', 'major', 'global', 'business', 'information', 'systems', 'hong', 'kong', 'university', 'science', 'technology', 'head', 'student', 'ambassador', 'time', 'university', 'data', 'ai', 'solution', 'specialist', 'ssp', 'company', 'microsoft', 'dates', 'employed', 'feb', '2018', 'present', 'employment', 'duration', '1', 'yr', '3', 'mos', 'location', 'hong', 'kong', 'believe', 'people', 'possible', 'mission', 'empower', 'person', 'organisation', 'planet', 'achieve', 'microsoft', 'ai', 'commercial', 'featuring', 'common', 'microsoft', 'ai', 'commercial', 'featurin', 'account', 'manager', 'company', 'meltwater', 'dates', 'employed', 'jul', '2017', 'feb', '2018', 'employment', 'duration', '8', 'mos', 'location', 'hong', 'kong', 'meltwater', 'helps', 'companies', 'better', 'informed', 'decisions', 'based', 'insights', 'outside', 'believe', 'business', 'strategy', 'increasingly', 'shaped', 'insights', 'online', 'data', '25,000', 'companies', 'use', 'meltwater', 'media', 'intelligence', 'platform', 'stay', 'billions', 'online', 'conversations', 'extract', 'relevant', 'real', 'time', 'insights', 'use', 'client', 'success', 'intern', 'company', 'meltwater', 'dates', 'employed', 'jun', '2016', 'aug', '2016', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'client', 'success', 'intern', 'duties', 'accomplishments', 'included', 'limited', 'learnt', 'executed', 'meltwater', 'sales', 'methodology', 'experiencing', 'tech', 'sales', 'cycle', 'whilst', 'engaging', 'c', 'level', 'executives', 'driving', 'strong', 'value', 'propositions', 'cu', 'hkust', 'chapter', 'co', 'founder', 'strategic', 'partnerships', 'director', 'company', '180', 'degrees', 'consulting', 'dates', 'employed', 'aug', '2015', 'jun', '2016', 'employment', 'duration', '11', 'mos', 'location', 'hong', 'kong', '180', 'degrees', 'provides', 'socially', 'conscious', 'organizations', 'world', 'high', 'quality', 'extremely', 'affordable', 'consulting', 'services', 'work', 'organizations', 'develop', 'innovative', 'practical', 'sustainable', 'solutions', 'challenges', 'facing', '180', 'degrees', 'connects', 'untapped', 'capabilities', 'university', 'students', 'unmet', 'needs', 'bank', 'wide', 'operations', 'intern', 'company', 'bank', 'china', 'hong', 'kong', 'dates', 'employed', 'jun', '2015', 'aug', '2015', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'executed', 'different', 'types', 'mortgage', 'loans', 'daily', 'basis', 'learnt', 'process', 'operations', 'involved', 'executing', 'mortgage', 'loan', 'conducted', 'diligence', '20', 'mortgage', 'cases', 'daily', 'involving', 'mortgage', 'agreements', 'individuals', 'smes', 'hong', 'kong', 'region', 'accounting', 'intern', 'company', 'lam', 'seng', 'plastics', 'sdn', 'bhd', 'dates', 'employed', 'mar', '2013', 'jun', '2013', 'employment', 'duration', '4', 'mos', 'location', 'kuala', 'lumpur', 'malaysia', 'lam', 'seng', 'plastics', 'industries', 'sdn', 'bhd', 'lsp', 'malaysia', 'plastic', 'products', 'plastic', 'household', 'ware', 'manufacturer', 'offering', 'high', 'quality', 'plastic', 'ware', 'home', 'industrial', 'use', 'plastic', 'oem', 'manufacturer', 'specializes', 'plastic', 'packaging', 'household', 'wares', 'industrial', 'containers', 'malaysia', 'plastic', 'products', 'household', 'wares', 'manufacturer', 'malaysia', 'plastic', 'products', 'household', 'data', 'ai', 'solution', 'specialist', 'ssp', 'company', 'microsoft', 'dates', 'employed', 'feb', '2018', 'present', 'employment', 'duration', '1', 'yr', '3', 'mos', 'location', 'hong', 'kong', 'account', 'manager', 'company', 'meltwater', 'dates', 'employed', 'jul', '2017', 'feb', '2018', 'employment', 'duration', '8', 'mos', 'location', 'hong', 'kong', 'client', 'success', 'intern', 'company', 'meltwater', 'dates', 'employed', 'jun', '2016', 'aug', '2016', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'hkust', 'chapter', 'co', 'founder', 'strategic', 'partnerships', 'director', 'company', '180', 'degrees', 'consulting', 'dates', 'employed', 'aug', '2015', 'jun', '2016', 'employment', 'duration', '11', 'mos', 'location', 'hong', 'kong', 'bank', 'wide', 'operations', 'intern', 'company', 'bank', 'china', 'hong', 'kong', 'dates', 'employed', 'jun', '2015', 'aug', '2015', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'accounting', 'intern', 'company', 'lam', 'seng', 'plastics', 'sdn', 'bhd', 'dates', 'employed', 'mar', '2013', 'jun', '2013', 'employment', 'duration', '4', 'mos', 'location', 'kuala', 'lumpur', 'malaysia', 'hkust', 'business', 'school', 'degree', 'bachelor', 'business', 'administration', 'bba', 'global', 'business', 'information', 'systems', 'entrepreneurship', 'dates', 'attended', 'expected', 'graduation', '2013', '2017', 'activities', 'societies', 'head', 'student', 'ambassador', 'dragon', 'boat', 'team', 'business', 'cohort', 'community', 'cohort', 'chief', 'residential', 'college', 'mentor', 'whu', 'otto', 'beisheim', 'school', 'management', 'degree', 'bachelor', 'science', 'bsc', 'field', 'study', 'entrepreneurship', 'entrepreneurial', 'studies', 'dates', 'attended', 'expected', 'graduation', '2016', '2016', 'sji', 'international', 'degree', 'ib', 'diploma', 'program', 'grade', '42/45', 'dates', 'attended', 'expected', 'graduation', '2011', '2012', 'activities', 'societies', 'badminton', 'team', 'captain', 'school', 'choir', 'volunteer', 'lee', 'ah', 'mo', 'old', 'folks', 'home', 'gift', 'love', 'occasional', 'break', 'dancer', 'group', 'leader', 'international', 'volunteer', 'company', 'chres', 'village', 'school', 'orphanage', 'cambodia', 'dates', 'volunteered', 'oct', '2011', 'volunteer', 'duration', '1', 'mo', 'cause', 'children', 'student', 'volunteer', 'company', 'sri', 'kdu', 'secondary', 'school', 'malaysia', 'dates', 'volunteered', 'mar', '2009', 'volunteer', 'duration', '1', 'mo', 'cause', 'education', 'insight', 'selling', 'issued', 'date', 'applicable', 'expiration', 'date', 'certification', 'license', 'issued', 'oct', '2018no', 'expiration', 'date']



['business', 'strategist', 'skilled', 'joint', 'marketing', 'partner', 'management', 'market', 'strategy', 'channel', 'strategy', 'channel', 'sales', 'developer', 'background', 'strong', 'business', 'development', 'professional', 'bs', 'focused', 'information', 'computer', 'science', 'university', 'california', 'irvine', 'business', 'strategist', 'skilled', 'joint', 'marketing', 'partner', 'management', 'market', 'strategy', 'channel', 'strategy', 'channel', 'sales', 'developer', 'background', 'strong', 'business', 'development', 'professional', 'bs', 'focused', 'information', 'computer', 'science', 'university', 'california', 'irvine', 'apcj', 'director', 'cloud', 'field', 'operations', 'company', 'f5', 'networks', 'dates', 'employed', 'feb', '2019', 'present', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'global', 'alliance', 'partner', 'development', 'company', 'amazon', 'web', 'services', 'dates', 'employed', 'jan', '2018', 'feb', '2019', 'employment', 'duratio

Different way to get similarity matrix 

**1. spacy_tokenizer + genism(tfidf)**

2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity

3. spacy_tokenizer + spacy similarity


In [0]:
import pprint
from gensim import corpora

# Lowercase each document, split it by white space and filter out stopwords
texts = [spacy_tokenizer(text) for text in profile_df["profile_detail"]]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]

In [9]:
dictionary = corpora.Dictionary(processed_corpus)
pprint.pprint(dictionary.token2id)

{'"i': 2404,
 '+2': 1536,
 '+852': 2324,
 '-experiences': 2138,
 '-familiar': 2139,
 '-proven': 2140,
 '-provided': 2141,
 '.hopkins&billion': 2621,
 '.net': 2338,
 '///we': 999,
 '/pr': 1728,
 '0': 2943,
 '000': 2015,
 '1': 0,
 '1,400': 2142,
 '1/16': 2215,
 '1/4': 2216,
 '10': 1,
 '100': 848,
 '11': 2,
 '12': 167,
 '13': 631,
 '14': 168,
 '140': 1359,
 '15': 737,
 '150': 2016,
 '16': 738,
 '16293507': 3108,
 '17': 169,
 '18': 804,
 '180': 3151,
 '19': 1212,
 '1983': 2376,
 '1984': 1769,
 '1986': 1770,
 '1987': 2377,
 '1988': 2378,
 '1989': 1671,
 '1991': 2262,
 '1992': 1870,
 '1993': 3,
 '1994': 1729,
 '1995': 4,
 '1996': 1213,
 '1997': 5,
 '1998': 384,
 '1999': 6,
 '2': 7,
 '20': 538,
 '200': 1588,
 '2000': 170,
 '2001': 385,
 '2001no': 2784,
 '2002': 171,
 '2003': 8,
 '2004': 386,
 '2004no': 387,
 '2005': 172,
 '2006': 9,
 '2007': 388,
 '2007no': 389,
 '2008': 173,
 '2008no': 1000,
 '2009': 390,
 '2009no': 2288,
 '2010': 391,
 '2010no': 392,
 '2011': 393,
 '2012': 137,
 '2012no': 7

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

Streaming output truncated to the last 5000 lines.
  (43, 3),
  (48, 13),
  (49, 10),
  (50, 10),
  (54, 2),
  (56, 2),
  (62, 2),
  (65, 4),
  (70, 1),
  (78, 4),
  (79, 4),
  (82, 1),
  (88, 13),
  (101, 2),
  (110, 2),
  (114, 4),
  (126, 2),
  (127, 1),
  (132, 5),
  (135, 2),
  (136, 6),
  (137, 5),
  (138, 2),
  (152, 3),
  (173, 1),
  (179, 9),
  (182, 5),
  (191, 1),
  (196, 3),
  (230, 1),
  (295, 2),
  (296, 3),
  (299, 2),
  (330, 3),
  (331, 1),
  (345, 3),
  (355, 4),
  (368, 1),
  (407, 3),
  (479, 6),
  (481, 1),
  (503, 6),
  (547, 2),
  (550, 1),
  (572, 1),
  (580, 2),
  (589, 1),
  (601, 2),
  (672, 3),
  (764, 1),
  (796, 6),
  (797, 3),
  (985, 1),
  (995, 1),
  (1040, 3),
  (1072, 2),
  (1237, 1),
  (1258, 1),
  (1296, 2),
  (1377, 3),
  (1705, 4),
  (1839, 1),
  (2541, 1),
  (2623, 3),
  (2643, 1),
  (2644, 3),
  (2645, 1),
  (2646, 9),
  (2647, 3),
  (2648, 2),
  (2649, 3),
  (2650, 3),
  (2651, 2),
  (2652, 2),
  (2653, 3),
  (2654, 2)],
 [(0, 7),
  (1, 3),
  (

In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 3), (1, 2), (2, 2), (3, 1), (4, 3), (5, 4), (6, 4), (7, 9), (8, 6), (9, 2), (10, 2), (11, 2), (12, 2), (13, 1), (14, 2), (15, 3), (16, 2), (17, 4), (18, 2), (19, 1), (20, 4), (21, 1), (22, 1), (23, 2), (24, 6), (25, 2), (26, 3), (27, 1), (28, 3), (29, 2), (30, 3), (31, 7), (32, 3), (33, 4), (34, 3), (35, 2), (36, 1), (37, 14), (38, 3), (39, 1), (40, 2), (41, 15), (42, 2), (43, 1), (44, 5), (45, 5), (46, 5), (47, 3), (48, 14), (49, 14), (50, 14), (51, 1), (52, 2), (53, 1), (54, 1), (55, 2), (56, 2), (57, 3), (58, 2), (59, 4), (60, 1), (61, 2), (62, 1), (63, 5), (64, 1), (65, 10), (66, 1), (67, 1), (68, 4), (69, 2), (70, 5), (71, 1), (72, 3), (73, 1), (74, 9), (75, 6), (76, 2), (77, 5), (78, 10), (79, 12), (80, 2), (81, 2), (82, 2), (83, 4), (84, 4), (85, 2), (86, 5), (87, 2), (88, 14), (89, 3), (90, 3), (91, 1), (92, 6), (93, 1), (94, 1), (95, 2), (96, 4), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 1),

In [12]:
from gensim import models

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc) 

[(0, 0.0069707017708570155), (1, 0.025789536969590795), (2, 0.025789536969590795), (3, 0.04673108591836377), (4, 0.11248102137059454), (5, 0.17158885395552742), (6, 0.10480772484786874), (7, 0.03134832974615791), (8, 0.15169487504353701), (9, 0.03489836452933049), (10, 0.024422389232550693), (11, 0.02069723908331592), (12, 0.01642354027299932), (13, 0.007727106955636017), (14, 0.050564958347845676), (15, 0.016011215660232756), (16, 0.019013297870105014), (17, 0.04752848752190442), (18, 0.030369954208770217), (19, 0.03749367379019818), (20, 0.1869243436734551), (21, 0.042897213488881855), (22, 0.03543934334013277), (23, 0.08579442697776371), (24, 0.23954058600463263), (25, 0.050564958347845676), (26, 0.04949059554727104), (27, 0.005947564976219524), (28, 0.18411611323563912), (29, 0.061372037745213034), (30, 0.11977029300231631), (31, 0.03876234739746961), (32, 0.1063180300203983), (33, 0.13463920544286512), (34, 0.046824253986996356), (35, 0.02188203579664439), (36, 0.04673108591836377

In [13]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf] 
lsi_model.print_topics(2)

[(0,
  '0.219*"endorsements" + 0.176*"date" + 0.149*"marketing" + 0.137*"microsoft" + 0.130*"title" + 0.124*"issued" + 0.121*"data" + 0.117*"expiration" + 0.116*"endorsement" + 0.113*"sales"'),
 (1,
  '-0.308*"date" + 0.288*"marketing" + -0.221*"credential" + -0.219*"issued" + -0.209*"expiration" + 0.177*"endorsements" + -0.168*"architect" + -0.167*"aws" + -0.160*"certified" + -0.125*"authority"')]

In [14]:
for doc, as_text in zip(corpus_lsi, profile_df["profile_detail"]):
    print(doc, as_text)

[(0, 0.22588949685156884), (1, 0.13039234638797345)] Business Strategist skilled in Joint Marketing, Partner Management, Go-to-market Strategy, Channel Strategy, and Channel Sales with a developer background. Strong business development professional with a BS focused in Information of Computer Science from University of California, Irvine.  Business Strategist skilled in Joint Marketing, Partner Management, Go-to-market Strategy, Channel Strategy, and Channel Sales with a developer background. Strong business development professional with a BS focused in Information of Computer Science from University of California, Irvine.  APCJ Director, Cloud Field Operations Company Name F5 Networks Dates Employed Feb 2019 – Present Employment Duration 3 mos Location Hong Kong   Global Alliance and Partner Development Company Name Amazon Web Services Dates Employed Jan 2018 – Feb 2019 Employment Duration 1 yr 2 mos Location Hong Kong   GM of Greater China and Business Development Director, APAC and

In [15]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi_model[corpus])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [16]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [17]:
# target_profile = input("something here: ")
target_profile = "Data Analyst Company Name  Dates Employed Apr 2018 – Present Employment Duration 1 yr 1 mo Location Hong Kong   Consultant Company Name EmblocSoft (Hong Kong) Limited Dates Employed Jul 2016 – Nov 2017 Employment Duration 1 yr 5 mos Location Hong Kong 1. Handling internal Hadoop Big Data system and preparing the environment for application testing.2. Contacting with clients, understanding their business requirements for the project, and aiding in the application design.3. Communicating with application developer oversea and performing testing on the application developed.4. Performing data analysis, comparing the performances between different algorithms and optimizing the predictive model.5. Developing data collection script (Web Crawler) and integrating data into internal Hadoop system. And providing daily maintenance for the source code and procedure. See less See more   Consultant Company Name EmblocSoft (Hong Kong) Limited Dates Employed Jul 2016 – Nov 2017 Employment Duration 1 yr 5 mos Location Hong Kong   Degree Name Bachelor of Science - BS Field Of Study Statistics Dates attended or expected graduation 2012 – 2016"
vec_bow = dictionary.doc2bow(spacy_tokenizer(target_profile))
vec_lsi = lsi_model[vec_bow]  # convert the query to LSI space
print(vec_lsi) # [(0, 4.325712071055434), (1, 0.10321814096913436)]

[(0, 4.886485292659973), (1, -0.1205560549199439)]


In [18]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
sims_list = list(enumerate(sims))
print(sims_list)  # print (document_number, document_similarity) 2-tuples
# [(0, 0.9292294), (1, 0.94229215), (2, 0.8801497), (3, 0.8481252), (4, 0.46180546), (5, 0.99026155), (6, 0.8361044), (7, 0.9167311), (8, 0.945788), (9, 0.9108537), (10, 0.939298), (11, 0.9707484), (12, 0.7081927), (13, 0.9962038), (14, 0.94031435), (15, 0.98867893), (16, 0.97373974), (17, 0.8565564), (18, 0.9002751), (19, 0.9998934), (20, 0.9601573), (21, 0.974343), (22, 0.9954696), (23, 0.9364472), (24, 0.9045476), (25, 0.9517342), (26, 0.9895474), (27, 0.5886535), (28, 0.9879773), (29, 0.96027374), (30, 0.8291013), (31, 0.90754724), (32, 0.9455353), (33, 0.9633571), (34, 0.9500553), (35, 0.846267), (36, 0.99816054), (37, 0.98294055), (38, 0.99668795), (39, 0.9865852), (40, 0.9786716), (41, 0.99327886), (42, 0.8590433), (43, 0.9965947), (44, 0.8914145), (45, 0.97324646), (46, 0.9771461), (47, 0.94867384), (48, 0.9801449), (49, 0.9872153), (50, 0.91296655), (51, 0.92789364), (52, 0.9975303), (53, 0.9887251), (54, 0.87935024), (55, 0.99988556), (56, 0.92909425), (57, 0.87746537), (58, 0.69031847), (59, 0.9979629), (60, 0.99192476), (61, 0.0), (62, 0.91734785), (63, 0.935356), (64, 0.98033553), (65, 0.8308765), (66, 0.0), (67, 0.99724686), (68, 0.94500184), (69, 0.9746252), (70, 0.9749831), (71, 0.99836344), (72, 0.9802869), (73, 0.9619486), (74, 0.94985235), (75, 0.92642504), (76, 0.99992585), (77, 0.91236526), (78, 0.994359), (79, 0.99995834), (80, 0.9754804), (81, 0.99993163), (82, 0.98865926), (83, 0.9098932), (84, 0.998446), (85, 0.9505327), (86, 0.9889571), (87, 0.7607491), (88, 0.88005006), (89, 0.9696953), (90, 0.9603296), (91, 0.6386689), (92, 0.8183906), (93, 0.93847984), (94, 0.97630805), (95, 0.999987), (96, 0.9674048), (97, 0.9973955), (98, 0.9565276), (99, 0.9948601)]

[(0, 0.92893004), (1, 0.94202083), (2, 0.8797652), (3, 0.84855366), (4, 0.46108744), (5, 0.9901485), (6, 0.83566016), (7, 0.9164075), (8, 0.9455249), (9, 0.9111874), (10, 0.93902), (11, 0.9705537), (12, 0.70876384), (13, 0.9962739), (14, 0.9400386), (15, 0.98855716), (16, 0.97355515), (17, 0.8569737), (18, 0.89992243), (19, 0.99988127), (20, 0.9599308), (21, 0.9741605), (22, 0.99554616), (23, 0.93616295), (24, 0.90489227), (25, 0.95148546), (26, 0.9894303), (27, 0.58799905), (28, 0.98785186), (29, 0.96004754), (30, 0.8286485), (31, 0.9072071), (32, 0.94579846), (33, 0.96313965), (34, 0.94980234), (35, 0.8466978), (36, 0.99820924), (37, 0.9827913), (38, 0.9967534), (39, 0.98645276), (40, 0.97850496), (41, 0.99318486), (42, 0.85862863), (43, 0.9965276), (44, 0.8910474), (45, 0.97306013), (46, 0.97731775), (47, 0.9484176), (48, 0.9799841), (49, 0.9870859), (50, 0.912636), (51, 0.92819506), (52, 0.9974731), (53, 0.98860365), (54, 0.87896454), (55, 0.9998974), (56, 0.92879456), (57, 0.87785

In [19]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

[(95, 0.99999076),
 (79, 0.9999506),
 (81, 0.9999407),
 (76, 0.9999353),
 (55, 0.9998974),
 (19, 0.99988127),
 (84, 0.9984006),
 (71, 0.99831676),
 (36, 0.99820924),
 (59, 0.9979109),
 (52, 0.9974731),
 (97, 0.99733675),
 (67, 0.9971864),
 (38, 0.9967534),
 (43, 0.9965276),
 (13, 0.9962739),
 (22, 0.99554616),
 (99, 0.9947778),
 (78, 0.99427277),
 (41, 0.99318486),
 (60, 0.99182177),
 (5, 0.9901485),
 (26, 0.9894303),
 (86, 0.9888368),
 (53, 0.98860365),
 (15, 0.98855716),
 (82, 0.9885374),
 (28, 0.98785186),
 (49, 0.9870859),
 (39, 0.98645276),
 (37, 0.9827913),
 (64, 0.98017544),
 (72, 0.9801266),
 (48, 0.9799841),
 (40, 0.97850496),
 (46, 0.97731775),
 (94, 0.97648287),
 (80, 0.9753019),
 (70, 0.97480285),
 (69, 0.9744437),
 (21, 0.9741605),
 (16, 0.97355515),
 (45, 0.97306013),
 (11, 0.9705537),
 (89, 0.9698927),
 (96, 0.9671995),
 (33, 0.96313965),
 (73, 0.961727),
 (90, 0.9601035),
 (29, 0.96004754),
 (20, 0.9599308),
 (98, 0.95629126),
 (25, 0.95148546),
 (85, 0.95028096),
 (34,

In [20]:
print(doc)
print("Top 5 similar profile: ")
for each in sims[0:5]:
    print(each,profile_df["profile_detail"][each[0]])

[(0, 0.21476256989226672), (1, 0.02790944396911763)]
Top 5 similar profile: 
(95, 0.99999076) I am a second-year . student at CUHK. Prior to that, I have spent 4 years in Sun Yat-Sen University (Guangzhou) and obtained my . from School of Data and Computer Science in 2016. I am interested in automatic information extraction and natural language text generation.  I am a second-year . student at CUHK. Prior to that, I have spent 4 years in Sun Yat-Sen University (Guangzhou) and obtained my . from School of Data and Computer Science in 2016. I am interested in automatic information extraction and natural language text generation.  Research Intern, NLP Group@Tencent AI Lab Company Name  Dates Employed Jan 2018 – Present Employment Duration 1 yr 4 mos Location      Research Intern, Knowledge Computing Group@MSRA Company Name  Dates Employed Jul 2015 – Jun 2016 Employment Duration 1 yr Location  Natural Language ProcessingMachine LearningKnowledge Mining See more   Teaching Assistant Company

Different way to get similarity matrix 

1. spacy_tokenizer + genism(tfidf)

**2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity**

3. spacy_tokenizer + spacy similarity


In [0]:
documents = [(" ").join(spacy_tokenizer(text)) for text in profile_df["profile_detail"]]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = TfidfVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=profile_df["profile_name"])

In [23]:
df

,000,01,02002246,10,100,10118009,11,1100,111,11558293,11901698,11939,12,121,13,130,132,13d,14,140,15,150,151,1535,16,16293507,17,175,178b,18,180,1828,18345031,18351997,18603397,1898,19,1914246,193,1974641,...,xforcecolloboration,xgd,xi,xl8ys57czgyf,xml,xtradb,xu,yahoo,yale,yan,yasen,yat,year,years,yi,yin,york,yosjdv,young,youngest,youth,youths,yr,yrs,yuan,yuilop,z01,zegal,zensoft,zhefei,zhejiang,zicklin,zjz,zone,zones,zwoop,zz,новосибирская,область,россия
profile_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abby Tang,0.000000,0.0,0.0,0.035775,0.0,0.000000,0.035775,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.036050,0.120167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ai Shinozaki,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.017017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dr Alan Chow MIET MIEEE MAUSIMM,0.000000,0.0,0.0,0.032717,0.0,0.000000,0.057254,0.0,0.000000,0.0,0.0,0.0,0.051134,0.000000,0.0,0.0,0.0,0.0,0.027766,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.014325,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.036552,0.020785,0.0,0.0,0.0,0.0,0.0,0.0,0.048072,0.0,0.021979,0.120885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abel Cheung,0.000000,0.0,0.0,0.000000,0.0,0.027155,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.018857,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.065099,0.065099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai autodata,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xin LI,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.078844,0.0,0.0,0.0,0.000000,0.078844,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.078844,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315378,0.093136,0.079442,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.105008,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wong Loretta,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.047926,0.063902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Worldmoney Blockchain News,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.354053,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0

In [24]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
sims_matrix = cosine_similarity(df, df)
print(sims_matrix)

[[1.         0.01542482 0.20143616 ... 0.16799212 0.1476735  0.18518347]
 [0.01542482 1.         0.01511693 ... 0.01474565 0.0063966  0.02797098]
 [0.20143616 0.01511693 1.         ... 0.17325747 0.14803296 0.21950924]
 ...
 [0.16799212 0.01474565 0.17325747 ... 1.         0.08843764 0.16158089]
 [0.1476735  0.0063966  0.14803296 ... 0.08843764 1.         0.11285665]
 [0.18518347 0.02797098 0.21950924 ... 0.16158089 0.11285665 1.        ]]


Different way to get similarity matrix 

1. spacy_tokenizer + genism(tfidf)

2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity

**3. spacy_tokenizer + spacy similarity**


In [25]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [26]:
nlp = spacy.load("en_core_web_lg")
token1 = nlp("i love dog.")
token5 = nlp("i love dog")
token6 = nlp("dog love me.")
token2 = nlp("i love dog?")
token3 = nlp("i hate dog.")
token4 = nlp("cat love dog.")
token7 = nlp("rabbit like carrot.")
token1.similarity
print(token1.vector_norm,token2.vector_norm,token3.vector_norm,token4.vector_norm,token5.vector_norm,token6.vector_norm,token7.vector_norm)
# 4.431595721931874 4.507516618461838 4.391567403021321 4.6381735996661915 5.044537372524752 4.369449490570207 3.8596050346561275

4.431595721931874 4.507516618461838 4.391567403021321 4.6381735996661915 5.044537372524752 4.369449490570207 3.8596050346561275


In [27]:
nlp = spacy.load("en_core_web_lg")

# get profile detail and tokenize them with spacy
documents = [(" ").join(spacy_tokenizer(text)) for text in profile_df["profile_detail"]] 
# target_profile = input("something here: ")
# no. 55 Kenny Zhou profile
target_profile = "Data Analyst Company Name  Dates Employed Apr 2018 – Present Employment Duration 1 yr 1 mo Location Hong Kong   Consultant Company Name EmblocSoft (Hong Kong) Limited Dates Employed Jul 2016 – Nov 2017 Employment Duration 1 yr 5 mos Location Hong Kong 1. Handling internal Hadoop Big Data system and preparing the environment for application testing.2. Contacting with clients, understanding their business requirements for the project, and aiding in the application design.3. Communicating with application developer oversea and performing testing on the application developed.4. Performing data analysis, comparing the performances between different algorithms and optimizing the predictive model.5. Developing data collection script (Web Crawler) and integrating data into internal Hadoop system. And providing daily maintenance for the source code and procedure. See less See more   Consultant Company Name EmblocSoft (Hong Kong) Limited Dates Employed Jul 2016 – Nov 2017 Employment Duration 1 yr 5 mos Location Hong Kong   Degree Name Bachelor of Science - BS Field Of Study Statistics Dates attended or expected graduation 2012 – 2016"
target_profile = (" ").join(spacy_tokenizer(target_profile))
target_token = nlp(target_profile)

sims_list = []
i = 0
for profile in documents:
    profile_token = nlp(profile)
    sims_list.append((i,profile_token.similarity(target_token)))
    i += 1
print(sims_list)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


[(0, 0.9374086449720327), (1, 0.6610590698705607), (2, 0.9031718298461465), (3, 0.9404091751002703), (4, 0.5874446602941784), (5, 0.9411113970110376), (6, 0.8925028411627305), (7, 0.9212922033197783), (8, 0.9052172822665918), (9, 0.9330228010851337), (10, 0.8499668723898665), (11, 0.9206891550272777), (12, 0.9339213081473415), (13, 0.9152292889051024), (14, 0.9343166444312216), (15, 0.8760344579731107), (16, 0.9521692315395159), (17, 0.9065043327974137), (18, 0.9203152019402331), (19, 0.9282912599160239), (20, 0.9315911314431493), (21, 0.8980448511484345), (22, 0.9176924735372386), (23, 0.9200673934717277), (24, 0.9587333399479779), (25, 0.9386103455649195), (26, 0.9346252967746633), (27, 0.7014388532497049), (28, 0.7793650535806954), (29, 0.9363637275119207), (30, 0.9097482908958509), (31, 0.9304637245214816), (32, 0.9089960066828986), (33, 0.9259233218998454), (34, 0.9425009664980473), (35, 0.9377769486569653), (36, 0.9254050115414761), (37, 0.9377480551375179), (38, 0.94290895371710

In [28]:
sims_list = sorted(sims_list, key=lambda item: -item[1])
print(sims_list)

[(59, 0.9703407681435177), (94, 0.9675099412356839), (24, 0.9587333399479779), (68, 0.9585059075064367), (69, 0.9568276714487044), (63, 0.954756712516059), (16, 0.9521692315395159), (51, 0.9517337387732472), (81, 0.9499428448085427), (80, 0.9476865992124198), (78, 0.9465803433113728), (99, 0.9460841657682321), (77, 0.944039877562629), (79, 0.9438945452048871), (82, 0.9435331098666494), (53, 0.9431618027329807), (38, 0.9429089537171065), (75, 0.9428009073916621), (34, 0.9425009664980473), (84, 0.9417130576039278), (5, 0.9411113970110376), (3, 0.9404091751002703), (25, 0.9386103455649195), (35, 0.9377769486569653), (37, 0.9377480551375179), (0, 0.9374086449720327), (67, 0.9369481310285124), (29, 0.9363637275119207), (96, 0.9356370406695663), (73, 0.9355667732932019), (57, 0.9349924669263514), (40, 0.9349322574972238), (26, 0.9346252967746633), (14, 0.9343166444312216), (12, 0.9339213081473415), (9, 0.9330228010851337), (70, 0.9322855969993851), (20, 0.9315911314431493), (31, 0.9304637245

In [29]:
print("Top 5 similar profile: ")
for each in sims_list[0:5]:
    print(each,profile_df["profile_detail"][each[0]])

Top 5 similar profile: 
(59, 0.9703407681435177)   AI / Digital Analyst Company Name BNP Paribas Dates Employed Jul 2018 – Present Employment Duration 10 mos Location Hong Kong   Research Assistant, School of Public Health Company Name The University of Hong Kong Dates Employed Jul 2017 – Jun 2018 Employment Duration 1 yr Location Hong Kong Developed a web-based visualization tool which converts phylogeny tree data into a graph See more   Data Analytics Intern Company Name DBS Bank Dates Employed Oct 2017 – May 2018 Employment Duration 8 mos Location Hong Kong Developed a visualization tool for sales practice monitoring in wealth management Developed a rule-based procurement fraud detection model See more   Consultant Intern Company Name Coherent Capital Advisors, Ltd Dates Employed Jan 2017 – Oct 2017 Employment Duration 10 mos Location Hong Kong Completed various web scrapping tasks for data mining Developed a premium calculation model for Investment-linked insurance See more   Actua